---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'limit_output': 5})

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    x = len(spam_data[spam_data['target'] == 1])
    y = len(spam_data['target'])
    
    return 100*x/y

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    res = max(vect.get_feature_names(), key = len)
    return res

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    clf = MultinomialNB(alpha = 0.1).fit(X_train_vectorized, y_train)
    X_test_vect= vect.transform(X_test)
    
    pred = clf.predict(X_test_vect)
    res = roc_auc_score(y_test, pred)
    print((y_test))
    print((pred))
    return res

In [8]:
answer_three()

4456    0
690     0
944     0
3768    0
1189    0
       ..
1889    0
2250    0
2915    0
1282    0
1206    1
Name: target, Length: 1393, dtype: int32
[0 0 0 ... 0 0 1]


0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    arr = X_train_vectorized.max(0).toarray()[0]
    feature_names = np.array(vect.get_feature_names())
    
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    
    largest = feature_names[sorted_tfidf_index[:20]]
    smallest = feature_names[sorted_tfidf_index[-20:]]
    
  
    serlarg = pd.Series(arr[sorted_tfidf_index[:20]],largest)
    sersmall = pd.Series(arr[sorted_tfidf_index[-20:]],smallest)

    
    serlarg = serlarg.sort_index()
    sersmall = sersmall.sort_index()
    
    serlarg.index.name=None
    sersmall.index.name=None

    
    return (serlarg,sersmall)

In [10]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 diwali          0.091250
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 mornings        0.091250
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 dtype: float64,
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 blank        0.932702
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 tick         0.980166
 too          1.000000
 where        1.000000
 yup          1.000000
 dt

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_five():

    vect = TfidfVectorizer(min_df=3).fit(X_train)  
    
    X_train_vectorized = vect.transform(X_train)
    
    clf = MultinomialNB(alpha = 0.1).fit(X_train_vectorized, y_train)
    X_test_vect= vect.transform(X_test)
    
    pred = clf.predict(X_test_vect)

    
    return roc_auc_score(y_test, pred)


In [12]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    
    x = spam_data[spam_data['target'] == 1]
    
    x['textlen'] = x['text'].str.len()
    
    bux = x['textlen'].sum(axis = 0, skipna = True)
    
    
    
    
    y = spam_data[spam_data['target'] == 0]
    
    y['textlen'] = y['text'].str.len()
   
    buy = y['textlen'].sum(axis = 0, skipna = True)
    
    return (buy/len(y) , bux/len(x))

In [14]:
answer_six()

<ipython-input-13-fae14f50cd0f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['textlen'] = x['text'].str.len()
<ipython-input-13-fae14f50cd0f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['textlen'] = y['text'].str.len()


(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC
import numpy as np

def answer_seven():
    
    spam_data['textlen'] = spam_data['text'].str.len()
    
    arr = (spam_data[['textlen']])
    
    X_trainn, X_testt, y_trainn, y_testt = train_test_split(spam_data['textlen'], 
                                                    spam_data['target'], 
                                                    random_state=0)
    
   
    vect = TfidfVectorizer(min_df=5).fit(X_train)  
    
    X_train_vectorized = vect.transform(X_train)
    
    X_train_vect = add_feature(X_train_vectorized, X_trainn)
    
    
    
    
    X_test_vectorized= vect.transform(X_test)
    
    X_test_vect = add_feature(X_test_vectorized, X_testt)
    

    

    clf = SVC(C = 10000)
    
    clf.fit(X_train_vect, y_train)
    
    res = clf.predict(X_test_vect)
    
  
    
    
    return roc_auc_score(y_test, res)

In [17]:
answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
    
    def count_digits(string):
        
        return sum(item.isdigit() for item in string)
        
    x = spam_data[spam_data['target'] == 1]
    
    x['dig'] = x['text'].apply(count_digits)
    
    bux = x['dig'].sum(axis = 0, skipna = True)
    
    
    
    
    y = spam_data[spam_data['target'] == 0]
    
    y['dig'] = y['text'].apply(count_digits)
   
    buy = y['dig'].sum(axis = 0, skipna = True)
    
    
    return (buy/len(y) , bux/len(x))
    


In [19]:
answer_eight()

<ipython-input-18-1e449e871343>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['dig'] = x['text'].apply(count_digits)
<ipython-input-18-1e449e871343>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['dig'] = y['text'].apply(count_digits)


(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.linear_model import LogisticRegression

def answer_nine(): 
    
    def count_digits(string):
        
        return sum(item.isdigit() for item in string)
    
    spam_data['dig'] = spam_data['text'].apply(count_digits)
    spam_data['textlen'] = spam_data['text'].str.len()
    X = spam_data[['dig','textlen']]
    
    
    X_trainn, X_testt, y_trainn, y_testt = train_test_split(X, 
                                                    spam_data['target'], 
                                                    random_state=0)
    
    
    vect = CountVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    
    X_train_vectorized = vect.transform(X_train)
    
    X_train_vect = add_feature(X_train_vectorized, X_trainn['dig'])
    X_train_vect = add_feature(X_train_vect, X_trainn['textlen'])
    
    
    
    
    X_test_vectorized = vect.transform(X_test)
    
    X_test_vect = add_feature(X_test_vectorized, X_testt['dig'])
    X_test_vect = add_feature(X_test_vect, X_testt['textlen'])
    
    


    clf = LogisticRegression(C = 100)
    
    clf.fit(X_train_vect, y_train)
   
    res = clf.predict(X_test_vect)
       
    
    return (res)
    

    


In [21]:
answer_nine()

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 0, 0, ..., 0, 0, 1])

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def answer_ten():
    
    spam_data['nonword'] = spam_data['text'].str.findall(r'[^a-zA-Z0-9 ]').str.len()
    
    x = spam_data[spam_data['target'] == 1]
    
    x['nonword'] = x['text'].str.findall(r'[\W]').str.len()

    bux = x['nonword'].sum(axis = 0, skipna = True)
    
    
    
    
    y = spam_data[spam_data['target'] == 0]
    
    y['nonword'] = y['text'].str.findall(r'[\W]').str.len()

    buy = y['nonword'].sum(axis = 0, skipna = True)
    

    return (buy/len(y),bux/len(x))
    


In [23]:
answer_ten()

<ipython-input-22-37b8da9c7705>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['nonword'] = x['text'].str.findall(r'[\W]').str.len()
<ipython-input-22-37b8da9c7705>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['nonword'] = y['text'].str.findall(r'[\W]').str.len()


(17.29181347150259, 29.041499330655956)